In [115]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [116]:
%config InlineBackend.figure_format = 'retina'

In [117]:
import numpy as np
import matplotlib.pyplot as plt
from model import *
import sys 
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import StepLR
from loss_function import *

In [118]:
data_model = diffusion_equation()
dataset = data_model.generate_training_data(500, 10, dlt_t = 0.001)

In [119]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam

data_loader = DataLoader(dataset, batch_size=64, shuffle=True)

# Define a simple model with a single trainable parameter `k`
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.k = nn.Parameter(torch.randn(()))  # Initialize k as a parameter

    def forward(self, lace_1, lace_2):
        g = lace_1
        h = self.k * lace_1 + lace_2
        return g, h

model = SimpleModel()

In [120]:
data_loader = DataLoader(dataset, batch_size=64, shuffle=True)
for batch in data_loader:
    x, y, lace_1, lace_2 = batch
    break

In [121]:
g, h = model(lace_1, lace_2)
x = x.reshape(-1, 1)
y = y.reshape(-1, 1)
g = g.reshape(-1, 1)
h = h.reshape(-1, 1)
gh = torch.cat([g, h], dim=1)
q, _ = torch.linalg.qr(gh)
print(gh.shape)
print(q.shape)

torch.Size([64, 2])
torch.Size([64, 2])


In [122]:
print(terminal_loss(q, y))

tensor(0.2459, grad_fn=<MseLossBackward0>)


In [123]:
lambda_1, lambda_2 = 1, 1
def loss_function(y, g, h):
    h_orthogonal = h / torch.linalg.norm(h, dim=1).reshape(-1, 1)
    gh = torch.cat([g, h], dim=1)
    q, _ = torch.linalg.qr(gh)
    return lambda_1 * terminal_loss(q, y) + lambda_2 * orthogonal_loss(g, h_orthogonal)

def train_one_epoch(model, optimizer, scheduler, train_loader, epoch):
    model.train()  # Set model to training mode
    total_loss = 0.0

    # Iterate over batches
    for batch_idx, (_, _, lace_1, lace_2) in enumerate(train_loader):
        optimizer.zero_grad()  # Clear gradients

        _, y, lace_1, lace_2 = batch

        # Compute g and h using your model. This is model-specific and might look different.
        g, h = model(lace_1, lace_2)

        # Compute the custom loss
        loss = loss_function(y, g, h)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # if batch_idx % 100 == 0:
            # print(f"Epoch {epoch+1}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}")

    scheduler.step()
    avg_loss = total_loss / len(train_loader)
    print(f"Training Epoch: {epoch+1}, Average Loss: {avg_loss:.4e}")
    return avg_loss


In [124]:
# Define optimizer
optimizer = Adam(model.parameters(), lr=0.001)
epochs = 500
scheduler = StepLR(optimizer, step_size=50, gamma=0.8)

In [128]:
train_loss_history = []
para_history = []
for epoch in range(epochs):
    train_loss = train_one_epoch(model, optimizer, scheduler, data_loader, epoch)
    train_loss_history.append(train_loss)
    para_history.append(model.k.clone().detach().numpy())


Epoch 1, Batch 0/28204, Loss: 0.5630
Epoch 1, Batch 100/28204, Loss: 0.5630
Epoch 1, Batch 200/28204, Loss: 0.5630
Epoch 1, Batch 300/28204, Loss: 0.5630
Epoch 1, Batch 400/28204, Loss: 0.5630
Epoch 1, Batch 500/28204, Loss: 0.5630
Epoch 1, Batch 600/28204, Loss: 0.5630
Epoch 1, Batch 700/28204, Loss: 0.5630
Epoch 1, Batch 800/28204, Loss: 0.5630
Epoch 1, Batch 900/28204, Loss: 0.5630
Epoch 1, Batch 1000/28204, Loss: 0.5630
Epoch 1, Batch 1100/28204, Loss: 0.5630
Epoch 1, Batch 1200/28204, Loss: 0.5630
Epoch 1, Batch 1300/28204, Loss: 0.5630
Epoch 1, Batch 1400/28204, Loss: 0.5630
Epoch 1, Batch 1500/28204, Loss: 0.5630
Epoch 1, Batch 1600/28204, Loss: 0.5630
Epoch 1, Batch 1700/28204, Loss: 0.5630
Epoch 1, Batch 1800/28204, Loss: 0.5630
Epoch 1, Batch 1900/28204, Loss: 0.5630
Epoch 1, Batch 2000/28204, Loss: 0.5630
Epoch 1, Batch 2100/28204, Loss: 0.5630
Epoch 1, Batch 2200/28204, Loss: 0.5630
Epoch 1, Batch 2300/28204, Loss: 0.5630
Epoch 1, Batch 2400/28204, Loss: 0.5630
Epoch 1, Bat

In [127]:
para_history = np.array(para_history)
print(para_history)

[]
